In [1]:
import sys 
import os 
import numpy as np
import pandas as pd
import sklearn as sk
import re
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from scipy.stats import ks_2samp
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

In [2]:
pd.set_option('display.max_columns', None)

In [5]:
os.chdir("C:/Users/Benedito/Desktop/MECAI/Introdução a Estatistica/MVP")

In [6]:
df = pd.read_csv("validacao_decision_tree_1510.csv")

C:\Anaconda3\envs\benedito\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (92) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,Unnamed: 0,SK_ID_CURR,score,y_pred,grupos,index,Unnamed: 0.1,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_start_dawn_previous,cnt_appl_start_morning_previous,cnt_appl_start_afternoon_previous,cnt_appl_start_night_previous,min_rate_down_payment_previous,max_interest_rate_primary_previous,max_interest_rate_privileged_previous,cnt_purpose_repair_previous,cnt_purpose_edu_bus_dev_previous,cnt_purpose_medicine_urgent_previous,cnt_purpose_superfluous_previous,cnt_purpose_real_estate_previous,cnt_purpose_house_needs_previous,cnt_purpose_car_previous,cnt_purpose_payment_previous,cnt_approved_contract_previous,cnt_canceled_previous,cnt_refused_previous,cnt_unused_previous,cnt_cash_bank_payment_previous,cnt_cashless_employer_payment_previous,cnt_noncash_account_payment_previous,cnt_unacompanied_suite_previous,cnt_family_suite_previous,cnt_spouse_suite_previous,cnt_group_people_suite_previous,cnt_children_suite_previous,max_client_type_previous,cnt_goods_leisure_previous,cnt_goods_electronic_previous,cnt_goods_vehicles_previous,cnt_goods_superfluous_previous,cnt_goods_medicine_previous,cnt_goods_homeware_previous,cnt_goods_construction_previous,cnt_goods_office_edu_previous,cnt_goods_weapon_previous,cnt_portfolio_cards_previous,cnt_portfolio_cash_previous,cnt_portfolio_pos_previous,cnt_portfolio_cars_previous,cnt_product_xsell_previous,cnt_product_walkin_previous,cnt_channel_regional_previous,cnt_channel_cash_previous,cnt_channel_contact_center_previous,cnt_channel_corporate_car_previous,cnt_seller_electronics_previous,cnt_seller_connectivity_previo

In [24]:
dash = pd.DataFrame()

In [25]:
dash['avg_income'] = df.groupby(['grupos', 'NAME_FAMILY_STATUS'])['AMT_INCOME_TOTAL'].mean()

In [26]:
dash['avg_amt_credit']=df.groupby(['grupos', 'NAME_FAMILY_STATUS'])['AMT_CREDIT'].mean()

In [27]:
dash['default_rate']=df.groupby(['grupos', 'NAME_FAMILY_STATUS'])['TARGET'].mean()

In [28]:
dash.reset_index(inplace=True)

In [29]:
dash

,grupos,NAME_FAMILY_STATUS,avg_income,avg_amt_credit,default_rate
0,grupo_1_default,Civil marriage,170927.856992,486419.453390,0.118644
1,grupo_1_default,Married,174260.832656,591919.726111,0.107501
2,grupo_1_default,Separated,173865.959559,520265.437500,0.120098
3,grupo_1_default,Single / not married,167968.867459,445213.709095,0.119287
4,grupo_1_default,Unknown,450000.000000,675000.000000,0.000000
5,grupo_1_default,Widow,132205.786853,447662.187251,0.087649
6,grupo_2_non_default,Civil marriage,156134.280146,502140.626539,0.142151
7,grupo_2_non_default,Married,159368.829136,595615.081430,0.118934
8,grupo_2_non_default,Separated,159880.049184,510251.180644,0.122242
9,grupo_2_non_default,Single / not married,153606.156311,464193.593133,0.148157


In [30]:
dash.to_csv("dash_1610.csv")